In [7]:
year1 =      "22"            # 만들 데이터의 전월 해 입력 ex) 20, 21, ...

mon1 =      "01"            # 만들 데이터의 전월 입력 ex) 03, 05, 08, 09, 10, 12 ,,,

year2 =     "22"           # 만들 데이터의 해 입력 ex) 20, 21, ...

mon2 =      "02"            # 만들 데이터의 현월 입력 ex) 03, 05, 08, 09, 10, 12 ,,,



In [8]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import openpyxl
import copy as c
os.chdir('C:/Users/tommy/Desktop/파이썬/가스 데이터/final_test/')
exclusion = './data/exclusion.xlsx'
base = './data/'+year1+'.'+mon1+'월 용기수불.xlsx'
file_name = './data/20'+year2+'.'+mon2+' 일일납품내역.xlsx'
file3 = './result_data/20'+year2+'.'+mon2+'월_거래처용기수불_pre.xlsx'

file_out1 = './result_data/'+year2+'.'+mon2+'월 용기수불.xlsx'

standard =pd.read_excel(base, skiprows = 2, sheet_name='거래처별용기수불')
standard_name =list(standard.columns[list(map(lambda x : 'Unnamed' not in x, standard.columns))])
standard_name = list(map(lambda x : x.upper(),standard_name))
standard_name = list(map(lambda x : re.sub(" ","",x),standard_name))

wb = openpyxl.load_workbook(file_name)
names = wb.sheetnames


# final_name : sheet name정리 
names = [i for i in names if i[:2] == mon2]
names1 = list(map(lambda x : re.sub(".+\\~{1}.+|(Sheet1)","",x),names))
final_name = [i for i in names1 if len(i) > 0]

# 필요한 데이터 가지고 오기 - 함수
def gas_car(date):
    df = pd.read_excel(file_name, skiprows = 2, sheet_name= date)
    df = df[['Date', 'Customer','GasCode', '실병하차','실병상차','공병하차', '공병상차','CarCode','Serial']]
    df['Customer'] = df['Customer'].apply(lambda x : re.sub(" ","",str(x)) if pd.notnull(x) else x)
    df['GasCode'] = df['GasCode'].apply(lambda x : x.upper() if pd.notnull(x) else x)
    df['GasCode'] = df['GasCode'].apply(lambda x : re.sub(" ","",x) if pd.notnull(x) else x)
    n = df[df['Date'] == '금일합계'].index[0]
    df = df.iloc[:n,:]

    return df

# gas_car()함수 적용하면서 모두 합치기 
final = pd.DataFrame()
for i in final_name:
    final = pd.concat([final,gas_car(i)])
    
    
final["실병하차"] = final["실병하차"].fillna(0)
final["공병하차"] = final["공병하차"].fillna(0)
final["공병상차"] = final["공병상차"].fillna(0)
final["실병상차"] = final["실병상차"].fillna(0)

final["실병하차"] = final["실병하차"] + final["공병하차"]
final["공병상차"] = final["공병상차"] + final["실병상차"]

# 'Customer','GasCode','실병하차','공병상차' 열만 가지고 오기
# 실병하차, 공병하차 데이터 행으로 합치기
final1 = final[['Customer','GasCode','실병하차','공병상차']]
final1_1 = final[['Customer','GasCode','실병하차']]
final1_1.rename(columns = {'실병하차':'value'},inplace = True)
final1_1['group'] = '실병하차'

final1_2 = final[['Customer','GasCode','공병상차']]
final1_2.rename(columns = {'공병상차':'value'},inplace = True)
final1_2['group'] = '공병상차'

final2 = pd.concat([final1_1,final1_2])

# 회사별 group별(실병하차, 공병하차) sum
ex_final = final2.pivot_table(index=['Customer'], columns=['GasCode','group'],
                          values='value',aggfunc = "sum").reset_index()
ex_final = ex_final.fillna(0)

# 가스 이름 갖고와서 공병상차 실병하차 붙히기
name1 = list(map(lambda x : x[0],ex_final.columns))[1:]
name2 = list(map(lambda x : x[1],ex_final.columns))[1:]
name_f = [name1[i] + "_" + name2[i] for i in range(len(name1))]


name_f = list(map(lambda x : re.sub("공병상차","3_공병상차",x),name_f))
name_f = list(map(lambda x : re.sub("실병하차","2_실병하차",x),name_f))

# 열순서대로 정렬
ex_final.columns= ['Customer'] + name_f
ex_final = ex_final[['Customer']+sorted(ex_final.columns[1:])]

df_r1 = pd.read_excel(file3)
df_r1 = df_r1[df_r1["Customer"] != 0]
df_r1['Customer'] = df_r1['Customer'].apply(lambda x : re.sub(" ","",str(x)) if pd.notnull(x) else x)

import re
A = [i for i in ex_final.columns if i[-4:] == "실병하차"]
A = list(map(lambda x : re.sub("_2_실병하차","",x),A))
B = [i for i in df_r1.columns if i[-4:] == "전월재고"][1:]
B = list(map(lambda x: re.sub("_1_전월재고","",x),B))

A_out = list(set(A)-set(B))
B_out = list(set(B)-set(A))

plus_A = []
plus_B = []
if len(A_out) != 0:
    plus_A = list(map(lambda x : x + "_" +'4_금월재고',A_out)) + list(map(lambda x : x + "_" +'1_전월재고',A_out))
if len(B_out) != 0:
    plus_B = list(map(lambda x : x + "_" +'2_실병하차',B_out)) + list(map(lambda x : x + "_" +'3_공병상차',B_out))
    
plus = plus_A + plus_B
if len(plus):
    for i in plus:
        ex_final[i] = 0
        
df_r2 = pd.merge(df_r1,ex_final,on = "Customer",how = "outer")
not_in_lastm = list(df_r2[pd.isnull(df_r2['합계_1_전월재고'])]["Customer"])
df_r3 = df_r2[(list(df_r2.columns)[:2] + sorted(df_r2.columns[2:]))].fillna(0)  

exclusion_data = pd.read_excel(exclusion)
df_r3 = pd.merge(pd.DataFrame({'Customer' : list(set(df_r3["Customer"]) - set(exclusion_data['termination']))}),df_r3, on = "Customer")

l1 = [i for i in df_r3.columns if i[-4:] == "실병하차"]
l2 = [i for i in df_r3.columns if i[-4:] == "공병상차"]
l3 = [i for i in df_r3.columns if (i[-4:] == "금월재고")]

    
for i in range(1,int(len(df_r3.columns)/4+1)):
    df_r3.iloc[:,4*i+1] = df_r3.iloc[:,4*i-2] + df_r3.iloc[:,4*i-1] - df_r3.iloc[:,4*i]


try:
    df_r3.insert(2, '합계_2_실병하차',0)
    df_r3.insert(3, '합계_3_공병상차',0)
    df_r3.insert(4, '합계_4_금월재고',0)
except:
    df_r3['합계_2_실병하차'] = 0
    df_r3['합계_3_공병상차'] = 0
    df_r3['합계_4_금월재고'] = 0
    
l4 = [i for i in df_r3.columns if (i[-4:] == "실병하차") and (i[:3] != "기화기")][1:]
for i in l4:
    df_r3['합계_2_실병하차'] += df_r3[i]
    
l5 = [i for i in df_r3.columns if (i[-4:] == "공병상차") and (i[:3] != "기화기")][1:]
for i in l5:
    df_r3['합계_3_공병상차'] += df_r3[i]
    
l6 = [i for i in df_r3.columns if (i[-4:] == "금월재고") and (i[:3] != "기화기")][1:]
for i in l6:
    df_r3['합계_4_금월재고'] += df_r3[i]
    
a = [i for i in df_r3.columns if i[-4:] == "전월재고"]
a_mod = list(map(lambda x : re.sub('_1_전월재고',"",x),a))[1:]
llll = []
for i in a_mod:
    llll.extend([i]*4)
    
def split_name(x):
    try:
        res = x.split('_')[2]
    except :
        res = x
    return (res)
    
col = list(df_r3.columns)
col1 = list(map(split_name, col))
    
df_r3.columns = [['Customer','합계','합계','합계','합계'] + llll,col1]
df_r4 = df_r3.loc[:,(['Customer','합계'], slice(None))]
for i in standard_name[2:]:
    tmp = df_r3.loc[:,(i, slice(None))]
    df_r4 = pd.concat([df_r4,tmp], axis = 1)

if len(list(set(a_mod) - set(standard_name[2:])))>0:
    tmp = df_r3.loc[:,(list(set(a_mod) - set(standard_name[2:])), slice(None))]
    df_r4 = pd.concat([df_r4,tmp], axis = 1)


import xlsxwriter
with pd.ExcelWriter(file_out1) as writer:
    writer.book = xlsxwriter.Workbook(file_out1)
    pd.DataFrame({'A' : [year2+'.'+mon2+'월 용기수불부',len(df_r4)]}).to_excel(writer, sheet_name="거래처별용기수불",header=None,index = None)
    df_r4.to_excel(writer, sheet_name="거래처별용기수불", startrow=2)
writer.save()

not_in_lastm

<ipython-input-8-5c1941f19f5f>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final1_1['group'] = '실병하차'
<ipython-input-8-5c1941f19f5f>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final1_2['group'] = '공병상차'


[]